In [2]:
! pip install --upgrade pandas numpy openpyxl
! pip install --upgrade names py-avataaars cairosvg Pillow

Requirement already up-to-date: pandas in /home/laszlo_szikszai/.local/lib/python3.8/site-packages (1.4.2)
Requirement already up-to-date: numpy in /home/laszlo_szikszai/.local/lib/python3.8/site-packages (1.22.4)
Requirement already up-to-date: openpyxl in /home/laszlo_szikszai/.local/lib/python3.8/site-packages (3.0.10)
Requirement already up-to-date: names in /home/laszlo_szikszai/.local/lib/python3.8/site-packages (0.3.0)
Requirement already up-to-date: py-avataaars in /home/laszlo_szikszai/.local/lib/python3.8/site-packages (1.1.2)
Requirement already up-to-date: cairosvg in /home/laszlo_szikszai/.local/lib/python3.8/site-packages (2.5.2)
Requirement already up-to-date: Pillow in /home/laszlo_szikszai/.local/lib/python3.8/site-packages (9.1.1)


In [3]:
import pandas as pd
import numpy as np
import names as ns
import datetime as dt
import random as rd
import py_avataaars as pa
import os
import glob
import shutil
import json
import PIL

In [4]:
n_people = 30
genders = [np.random.choice(['male', 'female']) for _ in range(n_people)]
names = [ns.get_full_name(gender=genders[i]) for i in range(n_people)]
people = tuple(zip(names, genders))
badge = 'Start Badge'
from_mail = 'The Community'
n_levels = 5
n_per_level = 5
profile_images = 'images/profile'
badge_images = 'images/badge'
base_badge_image = 'images/base/badge.png'

In [5]:
def clean_images(folder, ext):
  files = glob.glob(folder + '/*.' + ext)
  for f in files:
    os.remove(f)
clean_images(profile_images, 'png')
clean_images(profile_images, 'jpg')
clean_images(badge_images, 'png')

In [6]:
top_types = {
  'female': [t for t in pa.TopType if 'LONG_HAIR' in t.name],
  'male': [t for t in pa.TopType if 'SHORT_HAIR' in t.name]
}
def get_py_avatar_config(gender):
  return {
    'style': pa.AvatarStyle.CIRCLE,
    'skin_color': np.random.choice(pa.SkinColor),
    'hair_color': np.random.choice(pa.HairColor),
    'facial_hair_type': 
      pa.FacialHairType.DEFAULT if gender == 'female' else np.random.choice(pa.FacialHairType),
    'facial_hair_color': np.random.choice(pa.HairColor),
    'top_type': np.random.choice(top_types[gender]),
    'mouth_type': pa.MouthType.SMILE,
    'eye_type': pa.EyesType.HAPPY,
    'eyebrow_type': pa.EyebrowType.DEFAULT,
    'nose_type': pa.NoseType.DEFAULT,
    'accessories_type': pa.AccessoriesType.DEFAULT,
    'clothe_type': np.random.choice(pa.ClotheType),
    'clothe_color': np.random.choice(pa.Color),
    'clothe_graphic_type': np.random.choice(pa.ClotheGraphicType),
  }
  

In [7]:
df_names = []
for name, gender in people:
    level = np.random.randint(0, n_levels)
    n_badge = np.random.randint(level*n_per_level + 1, (level+1)*n_per_level + 1)
    # print(f"{name}: {n_badge} badges (level {level})")
    df_names.extend([name] * n_badge)
    
    profile_image = profile_images + '/' + name.replace(' ', '_') + '.png'
    avatar = pa.PyAvataaar(**get_py_avatar_config(gender))
    avatar.render_png_file(profile_image)
    
    png = PIL.Image.open(profile_image).convert('RGBA')
    background = PIL.Image.new('RGBA', png.size, (255, 255, 255))
    alpha_composite = PIL.Image.alpha_composite(background, png).convert('RGB')
    alpha_composite.save(profile_image.replace('.png', '.jpg'), 'JPEG', quality=80)
n_assignments = len(df_names)
df_badges = [badge] * n_assignments
df_from = [from_mail] * n_assignments

In [8]:
date_from, date_to = dt.date(2019,1,1), dt.date(2022,1,1)
dates = [date_from]
while date_from != date_to:
    date_from += dt.timedelta(days=1)
    dates.append(date_from)
df_dates = rd.choices(dates, k=n_assignments)

In [9]:
assignments = pd.DataFrame({
    'Date': df_dates,
    'From: email': df_from,
    'To: full name': df_names,
    'Badge': df_badges
})
assignments.sort_values('Date', inplace=True)
assignments.to_excel('input/badges.xlsx', index=None)

In [10]:
badge_config = {
  'name': badge,
  'image': '{config}/' + badge_images + '/' + badge + '.png',
  'levels': []
}
shutil.copy(base_badge_image, badge_images + '/' + badge + '.png')
for l in range(1, n_levels):
  badge_image = badge_images + '/' + badge + '_' + str(l) + '.png'
  shutil.copy(base_badge_image, badge_image)
  badge_config['levels'].append({
    'name': badge + ' #' + str(l+1),
    'image': '{config}/' + badge_image,
    'n': l * n_per_level,
  })
config = {
  'input': '{config}/input/badges.xlsx',
  'output': '{config}/output',
  'profileImages': '{config}/' + profile_images,
  'title': 'Example Dashboard',
  'header': 'Look at these nice badges...',
  'ignoreMissingImages': False,
  'showUnassignedBadges': False,
  'showProfileImages': True,
  'blacklist': [],
  'badges': [badge_config]
}
with open('config.json', 'w') as f:
  json.dump(config, f)

In [13]:
! cd ..
! npm i
! npm run generate -- --config example/config.json

m#########⠂⠂⠂⠂⠂⠂⠂⠂⠂) ⠏ idealTree: timing idealTree Completed in 218ms
> badge-dashboard@0.0.1 prepare
> husky install

husky - Git hooks installed

up to date, audited 388 packages in 1s

30 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities

> badge-dashboard@0.0.1 generate
> node bin/generate.js "--config" "example/config.json"

Generator initialized
Loading assignments from: /home/laszlo_szikszai/Repo/badge-dashboard/example/input/badges.xlsx
 - Loading /home/laszlo_szikszai/Repo/badge-dashboard/example/input/badges.xlsx
   -> found 358 assignment
Processing 358 assignments
Finding missing images
-> found 0 missing image(s)
Rendering dashboard
Saving dashboard to /home/laszlo_szikszai/Repo/badge-dashboard/example/output/badges.html
